## 特征选择
特征选择决定了用哪个特征划分特征空间，也就是在哪个维度划分数据。

特征选择使用信息增益作为衡量标准。
$$g(D, A) = H(D) - H(D|A)$$

表示：由于特征A而使得数据集D的分类不确定度的减少程度。

In [1]:
import numpy as np

In [2]:
# 例5.1
D = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 1, 1],
    [0, 1, 1, 0, 1],
    [0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0],
    [1, 0, 0, 1, 0],
    [1, 1, 1, 1, 1],
    [1, 0, 1, 2, 1],
    [1, 0, 1, 2, 1],
    [2, 0, 1, 2, 1],
    [2, 0, 1, 1, 1],
    [2, 1, 0, 1, 1],
    [2, 1, 0, 2, 1],
    [2, 0, 0, 0, 0],
])

In [3]:
X = D[:, :-1]
y = D[:, -1]

In [4]:
X

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 1, 1, 1],
       [1, 0, 1, 2],
       [1, 0, 1, 2],
       [2, 0, 1, 2],
       [2, 0, 1, 1],
       [2, 1, 0, 1],
       [2, 1, 0, 2],
       [2, 0, 0, 0]])

In [5]:
y

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [6]:
from collections import Counter
y_conter = Counter(y)
y_conter

Counter({0: 6, 1: 9})

In [7]:
y_conter.most_common(1)[0][0]

1

In [8]:
y_conter.values()

dict_values([6, 9])

In [9]:
from collections import Counter
def cal_empirical_entropy(y):
    '''计算经验熵'''
    ret = 0.0
    y_counter = Counter(y)
    # v : |C_k|
    # len(y): |D|
    for v in y_counter.values():          
        ret += (v / len(y)) * np.log2(v / len(y))
    
    return -ret

In [10]:
cal_empirical_entropy(y)

0.9709505944546686

In [11]:
y[X[:, 0] == 0]

array([0, 0, 1, 1, 0])

In [12]:
def cal_conditional_entropy(X, y, dim, cal_empirical_entropy_=None):
    '''计算条件熵'''
    ret = 0.0
    x_counter = Counter(X[:, dim])
    # x_i: X的第dim个维度的第i个特征值
    # x_c: x_i的统计结果
    if cal_empirical_entropy_ is not None:
        for x_i, x_c in zip(x_counter.keys(), x_counter.values()):
            y_k = y[X[:, dim] == x_i]
            ret += (x_c / len(y)) * cal_empirical_entropy_(y_k)
    else:
        for x_c in x_counter.values():
            ret -= (x_c / len(y)) * np.log2(x_c / len(y))
            
    return ret

In [13]:
cal_conditional_entropy(X, y, 0)

1.584962500721156

In [14]:
def cal_information_gan(X, y, dim):
    '''计算信息增益'''
    return cal_empirical_entropy(y) - cal_conditional_entropy(X, y, dim, cal_empirical_entropy)

In [15]:
def cal_information_gan_ratio(X, y, dim):
    gan = cal_information_gan(X, y, dim)
    h = cal_conditional_entropy(X, y, dim)

    # 处理 h == 0
    if abs(h) < 1e-9:
        return 0
    return gan / h

In [16]:
cal_information_gan(X, y, 0)

0.08300749985576883

In [17]:
cal_information_gan(X, y, 1)

0.32365019815155627

In [18]:
cal_information_gan(X, y, 2)

0.4199730940219749

In [19]:
cal_information_gan(X, y, 3)

0.36298956253708536

## 决策树的生成

由于C4.5和ID3过程一致，所以采用C4.5算法，使用信息增益比实现，避免直接使用信息增益划分训练集时，偏向选择取值较多的特征的问题

In [20]:
class Tree:
    def __init__(self):
        self.sub = {}
        self.leaf = None
        self.condition = lambda x_i: x_i

    def __repr__(self):
        s = 'sub_node: {}, leaf:{}'.format(str(list(self.sub.keys())), self.leaf)
        return s

In [21]:
def build_tree(X, y, epsilon):
    assert len(X) == len(y), "X and y len error"
    
    node = Tree()  # 创建树的根节点
    y_counter = Counter(y)  # 对 label做一个统计
    
    # 1. 判断所有实例是否属于同一类， 设置node为点节点数
    # 2. 特征集是否是空集， 设置node为点节点数
    if len(y_counter) == 1 or len(X) == 0:
        node.leaf = y_counter.most_common(1)[0][0]
    
    else:
        # 3. 计算各个维度的信息增益比，找到最好的划分维度
        best_ratio = float('-inf')
        best_dim = None
        for dim in range(X.shape[-1]):
            
            gan_ratio = cal_information_gan_ratio(X, y, dim)
            if gan_ratio > best_ratio:
                best_ratio = gan_ratio
                best_dim = dim
                
        # 4. 如果best_ratio 小于ε， 设置T为点节点数
        if best_ratio < epsilon:
            node.leaf = y_counter.most_common(1)[0][0]
        else:
            x_counter = Counter(X[:, best_dim])
        
        # 5. 对best_dim下的每个特征值，将D划分为子空间，并构建子节点
            for x_i in x_counter:
                sub_X = X[X[:, best_dim] == x_i]
                sub_y = y[X[:, best_dim] == x_i]
                index = node.condition(x_i)
                node.sub[index] = build_tree(sub_X, sub_y, epsilon)
    
    return node

In [22]:
t = build_tree(X, y, 1e-2)

In [23]:
t.leaf

In [24]:
t.sub

{0: sub_node: [0, 1], leaf:None, 1: sub_node: [], leaf:1}

In [25]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [28]:
dt_clf.predict(X[:5])

array([0, 0, 1, 1, 0])

In [30]:
a = np.array([3, 2])
b = a
a * b

array([9, 4])